In [1]:
!pip install rdkit
!pip install optuna
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.8 MB/s eta 0:00:00


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import metrics

from xgboost import XGBClassifier
import time
import optuna

In [4]:
# upload this file to collab or clone this repo and add `data` directory with data
melanin_df = pd.read_csv('./data/melanin.csv')

In [5]:
melanin_df.head()

,SMILES,Class
0,CCN(CC)CCNC(=O)c1ccc(cc1)N.Cl,1
1,COCCNC(=O)CN1C2CCC1CC(C2)(c3cccnc3)O,1
2,CC1=NN=C(c2cc3c(cc2C1)OCO3)c4ccc(cc4)N,1
3,CC1C2Cc3ccc(cc3C1(CCN2CC=C)C)O,1
4,COc1ccc(cc1)c2coc3cc(ccc3c2=O)OC,1


In [6]:
melanin_df.Class.value_counts()

Class
1    607
0    173
Name: count, dtype: int64

In [7]:
descriptor_list = Descriptors.descList
descriptors = []

for descriptor in descriptor_list:
      descriptors.append(descriptor[0])
def get_descriptor_values(mol, descriptors):
    calc = MolecularDescriptorCalculator(descriptors)
    ds = calc.CalcDescriptors(mol)
    return ds[0]
for i in descriptors:
    melanin_df[i] = pd.Series(np.array([get_descriptor_values(Chem.MolFromSmiles(j), [i]) for j in melanin_df["SMILES"]]), index=melanin_df.index)

/tmp/ipykernel_342134/2275780979.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  melanin_df[i] = pd.Series(np.array([get_descriptor_values(Chem.MolFromSmiles(j), [i]) for j in melanin_df["SMILES"]]), index=melanin_df.index)
/tmp/ipykernel_342134/2275780979.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  melanin_df[i] = pd.Series(np.array([get_descriptor_values(Chem.MolFromSmiles(j), [i]) for j in melanin_df["SMILES"]]), index=melanin_df.index)
/tmp/ipykernel_342134/2275780979.py:11: PerformanceWarning: DataFrame is hig

In [8]:
melanin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Columns: 212 entries, SMILES to fr_urea
dtypes: float64(106), int64(105), object(1)
memory usage: 1.3+ MB


In [9]:
single_value_columns = melanin_df.columns[melanin_df.nunique() == 1]
melanin_df[single_value_columns]

,NumRadicalElectrons,SMR_VSA8,SlogP_VSA9,fr_aldehyde,fr_azide,fr_azo,fr_barbitur,fr_diazo,fr_isocyan,fr_isothiocyan,fr_lactam,fr_nitroso,fr_phos_acid,fr_phos_ester,fr_prisulfonamd,fr_thiocyan
0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
776,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
777,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
melanin_df.drop(columns=single_value_columns, inplace=True)

In [11]:
corr_matrix = melanin_df.drop(columns=['SMILES', 'Class']).corr().abs()

In [12]:
corr_matrix

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
MaxAbsEStateIndex,1.000000,1.000000,0.549487,0.517814,0.148163,0.145336,0.368054,0.374870,0.368826,0.385975,...,0.068537,0.057079,0.132178,0.073312,0.036169,0.012941,0.022827,0.033263,0.014641,0.094043
MaxEStateIndex,1.000000,1.000000,0.549487,0.517814,0.148163,0.145336,0.368054,0.374870,0.368826,0.385975,...,0.068537,0.057079,0.132178,0.073312,0.036169,0.012941,0.022827,0.033263,0.014641,0.094043
MinAbsEStateIndex,0.549487,0.549487,1.000000,0.364339,0.034567,0.103524,0.278091,0.278769,0.277457,0.265366,...,0.067192,0.063719,0.070668,0.033836,0.033334,0.008988,0.079464,0.116182,0.017031,0.024510
MinEStateIndex,0.517814,0.517814,0.364339,1.000000,0.167352,0.111044,0.405698,0.419271,0.406025,0.381325,...,0.017349,0.066786,0.493778,0.256334,0.013940,0.005433,0.035509,0.025377,0.008451,0.031224
qed,0.148163,0.148163,0.034567,0.167352,1.000000,0.176124,0.490615,0.495097,0.491267,0.487543,...,0.146499,0.055386,0.008351,0.019270,0.024371,0.024964,0.036488,0.010505,0.329915,0.024415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fr_tetrazole,0.012941,0.012941,0.008988,0.005433,0.024964,0.011923,0.022276,0.025766,0.022334,0.018236,...,0.005913,0.013489,0.010123,0.005913,0.003150,1.000000,0.011071,0.075074,0.014392,0.009275
fr_thiazole,0.022827,0.022827,0.079464,0.035509,0.036488,0.073606,0.023909,0.010955,0.023820,0.064786,...,0.016955,0.038678,0.099615,0.016955,0.009034,0.011071,1.000000,0.054805,0.041269,0.026595
fr_thiophene,0.033263,0.033263,0.116182,0.025377,0.010505,0.062258,0.035183,0.022801,0.035300,0.092893,...,0.022151,0.000195,0.090126,0.036617,0.011802,0.075074,0.054805,1.000000,0.040476,0.041169
fr_unbrch_alkane,0.014641,0.014641,0.017031,0.008451,0.329915,0.086923,0.076669,0.051938,0.076973,0.129244,...,0.274662,0.027972,0.019777,0.022041,0.011744,0.014392,0.041269,0.040476,1.000000,0.008012


In [13]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

melanin_df.drop(to_drop, axis=1, inplace=True)

In [14]:
melanin_df.head()

,SMILES,Class,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,MaxPartialCharge,MinPartialCharge,...,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCN(CC)CCNC(=O)c1ccc(cc1)N.Cl,1,11.743677,0.000000,-0.044300,0.775469,9.944444,271.792,0.250826,-0.398728,...,0,0,0,0,0,0,0,0,0,0
1,COCCNC(=O)CN1C2CCC1CC(C2)(c3cccnc3)O,1,12.055486,0.033118,-0.816349,0.753573,30.347826,319.405,0.233779,-0.384987,...,0,0,0,0,0,0,0,0,1,0
2,CC1=NN=C(c2cc3c(cc2C1)OCO3)c4ccc(cc4)N,1,5.773582,0.263537,0.263537,0.821909,15.681818,293.326,0.230801,-0.453584,...,0,0,0,0,0,0,0,0,0,0
3,CC1C2Cc3ccc(cc3C1(CCN2CC=C)C)O,1,9.798686,0.203036,0.203036,0.822957,33.789474,257.377,0.115392,-0.507956,...,0,0,0,0,0,0,0,0,0,0
4,COc1ccc(cc1)c2coc3cc(ccc3c2=O)OC,1,12.550924,0.065676,-0.065676,0.737634,10.571429,282.295,0.199993,-0.496768,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = melanin_df.drop(columns=['SMILES', 'Class'])
y = melanin_df['Class']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1488)

In [17]:
metric = 'auc'
base_params = {
    'objective': 'binary:logistic',
    'eval_metric': metric,
    'enable_categorical': True,
}

In [18]:
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

In [19]:
def objective(trial):
    params = {
        'tree_method': trial.suggest_categorical('tree_method', ['approx', 'hist']),
        'max_depth': trial.suggest_int('max_depth', 5, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 12),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10000, 10000),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 50, 50),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', scale_pos_weight, scale_pos_weight)
    }
    params.update(base_params)

    # Add pruning callback
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, f'validation_1-{metric}')

    # Initialize XGBClassifier
    model = XGBClassifier(callbacks=[pruning_callback], **params)

    # Train the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=0
    )

    # Save the best iteration for reference
    trial.set_user_attr('best_iteration', model.best_iteration)

    # Return the validation score
    return model.best_score

In [22]:
sampler = optuna.samplers.TPESampler(seed=1488)
study = optuna.create_study(direction='maximize', sampler=sampler)
tic = time.time()
while time.time() - tic < 100:
    study.optimize(objective, n_trials=1)

[I 2025-03-30 02:40:58,606] A new study created in memory with name: no-name-c5f53b98-93a6-4279-8b0f-9b581589331d
[I 2025-03-30 02:41:05,163] Trial 0 finished with value: 0.873435655253837 and parameters: {'tree_method': 'approx', 'max_depth': 10, 'min_child_weight': 8, 'subsample': 0.8889393258968677, 'colsample_bytree': 0.8258044087891845, 'reg_lambda': 0.05023256449955142, 'n_estimators': 10000, 'early_stopping_rounds': 50, 'scale_pos_weight': 0.2839506172839506}. Best is trial 0 with value: 0.873435655253837.
[I 2025-03-30 02:41:13,201] Trial 1 finished with value: 0.8369539551357733 and parameters: {'tree_method': 'approx', 'max_depth': 7, 'min_child_weight': 12, 'subsample': 0.47368572541632725, 'colsample_bytree': 0.5275802773931773, 'reg_lambda': 0.14804210086916686, 'n_estimators': 10000, 'early_stopping_rounds': 50, 'scale_pos_weight': 0.2839506172839506}. Best is trial 0 with value: 0.873435655253837.
[I 2025-03-30 02:41:18,461] Trial 2 finished with value: 0.838016528925619

In [23]:
print(f'best score = {study.best_trial.value}')
print('boosting params ---------------------------')
print(f'best boosting round: {study.best_trial.user_attrs["best_iteration"]}')
print('best tree params --------------------------')
for k, v in study.best_trial.params.items():
    print(k, ':', v)

best score = 0.880047225501771
boosting params ---------------------------
best boosting round: 13
best tree params --------------------------
tree_method : hist
max_depth : 10
min_child_weight : 6
subsample : 0.8832189875922608
colsample_bytree : 0.9975624205287873
reg_lambda : 0.023953657047454315
n_estimators : 10000
early_stopping_rounds : 50
scale_pos_weight : 0.2839506172839506


In [39]:
best_trial = XGBClassifier(**base_params, **study.best_trial.params)

best_trial.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=0
)

y_true = y_test
y_pred = best_trial.predict(X_test)
y_score = best_trial.predict_proba(X_test)[:,1]


print(metrics.classification_report(y_true, y_pred))
metrics.roc_auc_score(y_true, y_score)

              precision    recall  f1-score   support

           0       0.58      0.74      0.65        35
           1       0.92      0.84      0.88       121

    accuracy                           0.82       156
   macro avg       0.75      0.79      0.76       156
weighted avg       0.84      0.82      0.83       156



0.880047225501771

### Save model to the file

In [25]:
# config is a local config.py file strored in the root of this direcotry
from config import MODELS_SAVE_PATH

best_trial.save_model(MODELS_SAVE_PATH.joinpath('melanin_binding.json'))

### Test load model from the file

In [41]:
from config import MODELS_SAVE_PATH
from xgboost import XGBClassifier

xgbc = XGBClassifier()

model_path = MODELS_SAVE_PATH.joinpath('melanin_binding.json')
xgbc.load_model(model_path)

In [ ]:
# assuming data is loaded!
y_true = y_test
y_pred = xgbc.predict(X_test)
y_score = xgbc.predict_proba(X_test)[:,1]


print(metrics.classification_report(y_true, y_pred))
metrics.roc_auc_score(y_true, y_score)

              precision    recall  f1-score   support

           0       0.58      0.74      0.65        35
           1       0.92      0.84      0.88       121

    accuracy                           0.82       156
   macro avg       0.75      0.79      0.76       156
weighted avg       0.84      0.82      0.83       156



0.880047225501771